# Sioux Falls example

In [1]:
import sys
sys.path.append('../..')

## File paths

In [2]:
fldr = 'D:/release/Sample models/sioux_falls_2020_02_15'

# remove the comments for the lines below to run the Chicago model example instead
# fldr = 'D:/release/Sample models/Chicago_2020_02_15'
# proj_name = 'chicagomodel.sqlite'

dt_fldr = '0_tntp_data'
prj_fldr = '1_project'
new_proj_folder = '1_project_temp'

skm_fldr = '2_skim_results'
assg_fldr = '4_assignment_results'
dstr_fldr = '5_distribution_results'
frcst_fldr = '6_forecast'
ftr_fldr = '7_future_year_assignment'

## We copy the project to a different folder so we don't overwrite things like the matrix table

In [3]:
from shutil import copytree, rmtree
from os.path import isfile, isdir, join

if isdir(join(fldr, new_proj_folder)):
    rmtree(join(fldr, new_proj_folder))

copytree(join(fldr, prj_fldr), join(fldr, new_proj_folder))

'D:/release/Sample models/sioux_falls_2020_02_15\\1_project_temp'

## Opening the project

In [4]:
# Imports
from aequilibrae.project import Project
from os.path import join

In [5]:
project = Project()
project.load(join(fldr, new_proj_folder))

## Path computation

In [6]:
# imports
from aequilibrae.paths import PathResults, path_computation

In [7]:
# we build all graphs
project.network.build_graphs()
# We get warnings that several fields in the project are filled with NaNs.  Which is true, but we won't use those fields

In [8]:
# we grab the graph for cars
graph = project.network.graphs['c']

# let's say we want to minimize distance
graph.set_graph('distance')

# And will skim time and distance while we are at it
graph.set_skimming(['free_flow_time', 'distance'])

# And we will allow paths to be compute going through other centroids/centroid connectors
# required for the Sioux Falls network, as all nodes are centroids
graph.set_blocked_centroid_flows(False)

# instantiate a path results object and prepare it to work with the graph
res = PathResults()
res.prepare(graph)

# compute a path from node 2 to 13
path_computation(2, 13, graph, res)

In [9]:
# We can get the sequence of nodes we traverse
res.path_nodes

array([ 2,  1,  3, 12, 13], dtype=int64)

In [10]:
# We can get the link sequence we traverse
res.path

array([ 3,  2,  7, 37], dtype=int64)

In [11]:
# We can get the mileposts for our sequence of nodes
res.milepost

array([ 0.,  6., 10., 14., 17.])

In [12]:
# If we want to compute the path for a different destination and same origin, we can just do this
# It is way faster when you have large networks
res.update_trace(4)

In [13]:
res.path_nodes

array([2, 6, 5, 4], dtype=int64)

## Skimming

In [14]:
from aequilibrae.matrix import AequilibraeData, AequilibraeMatrix
from aequilibrae.paths import NetworkSkimming, SkimResults

In [15]:
# from before
graph = project.network.graphs['c'] # we grab the graph for cars
graph.set_graph('free_flow_time') # let's say we want to minimize time
graph.set_skimming(['free_flow_time', 'distance']) # And will skim time and distance
graph = project.network.graphs['c'] # we grab the graph for cars
graph.set_blocked_centroid_flows(False)

In [16]:
# And run the skimming
skm = NetworkSkimming(graph)
skm.execute()

In [21]:
# The result is an AequilibraEMatrix object
skims = skm.results.skims

# Which we can manipute directly from its temp file, if we wish
skims.matrices

memmap([[[ 0.,  0.],
         [ 6.,  6.],
         [ 4.,  4.],
         ...,
         [20., 20.],
         [17., 17.],
         [15., 15.]],

        [[ 6.,  6.],
         [ 0.,  0.],
         [10., 10.],
         ...,
         [21., 21.],
         [23., 23.],
         [21., 21.]],

        [[ 4.,  4.],
         [10., 10.],
         [ 0.,  0.],
         ...,
         [16., 16.],
         [13., 13.],
         [11., 11.]],

        ...,

        [[20., 20.],
         [21., 21.],
         [16., 16.],
         ...,
         [ 0.,  0.],
         [ 4.,  4.],
         [ 5.,  5.]],

        [[17., 17.],
         [23., 23.],
         [13., 13.],
         ...,
         [ 4.,  4.],
         [ 0.,  0.],
         [ 2.,  2.]],

        [[15., 15.],
         [21., 21.],
         [11., 11.],
         ...,
         [ 5.,  5.],
         [ 2.,  2.],
         [ 0.,  0.]]])

In [22]:
# Ore access each matrix
skims.free_flow_time

memmap([[ 0.,  6.,  4.,  8., 10., 11., 16., 13., 15., 18., 14.,  8., 11.,
         18., 23., 18., 20., 18., 22., 22., 18., 20., 17., 15.],
        [ 6.,  0., 10., 11.,  9.,  5., 10.,  7., 14., 16., 17., 14., 17.,
         21., 19., 12., 14., 12., 16., 16., 22., 21., 23., 21.],
        [ 4., 10.,  0.,  4.,  6., 10., 15., 12., 11., 14., 10.,  4.,  7.,
         14., 19., 17., 19., 17., 21., 20., 14., 16., 13., 11.],
        [ 8., 11.,  4.,  0.,  2.,  6., 11.,  8.,  7., 10.,  6.,  8., 11.,
         10., 15., 13., 15., 13., 17., 17., 18., 18., 14., 15.],
        [10.,  9.,  6.,  2.,  0.,  4.,  9.,  6.,  5.,  8.,  8., 10., 13.,
         12., 14., 11., 13., 11., 15., 15., 19., 17., 16., 17.],
        [11.,  5., 10.,  6.,  4.,  0.,  5.,  2.,  9., 11., 12., 14., 17.,
         16., 14.,  7.,  9.,  7., 11., 11., 17., 16., 20., 20.],
        [16., 10., 15., 11.,  9.,  5.,  0.,  3., 12.,  9., 14., 19., 19.,
         17., 12.,  5.,  7.,  2.,  9.,  6., 12., 11., 15., 15.],
        [13.,  7., 12.,  8.

In [ ]:
# We can save it to the project if we want
skm.save_to_project('base_skims')

# We can also retrieve this skim record to write something to its description
matrices = project.matrices
mat_record = matrices.get_record('base_skims')
mat_record.description = 'minimized FF travel time while also skimming distance'
mat_record.save()

# Traffic assignment with skimming

In [ ]:
from aequilibrae.matrix import AequilibraeMatrix
from aequilibrae.paths import TrafficAssignment, TrafficClass
from aequilibrae import logger
import logging

In [ ]:
# from before
graph = project.network.graphs['c'] # we grab the graph for cars
graph.set_graph('free_flow_time') # let's say we want to minimize time
graph.set_skimming(['free_flow_time', 'distance']) # And will skim time and distance
graph.set_blocked_centroid_flows(False)

In [ ]:
import sys
# Because assignment takes a long time, we want the log to be shown here
stdout_handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter("%(asctime)s;%(name)s;%(levelname)s ; %(message)s")
stdout_handler.setFormatter(formatter)
logger.addHandler(stdout_handler)

In [ ]:
demand = AequilibraeMatrix()
demand.load(join(fldr, dt_fldr, 'demand.omx'))
demand.computational_view(['matrix']) # We will only assign one user class stored as 'matrix' inside the OMX file

assig = TrafficAssignment()

# Creates the assignment class
assigclass = TrafficClass(graph, demand)

# The first thing to do is to add at list of traffic classes to be assigned
assig.add_class(assigclass)
# assig.set_classes([assigclass])

assig.set_vdf("BPR")  # This is not case-sensitive # Then we set the volume delay function

assig.set_vdf_parameters({"alpha": "b", "beta": "power"}) # And its parameters

assig.set_capacity_field("capacity") # The capacity and free flow travel times as they exist in the graph
assig.set_time_field("free_flow_time")

# And the algorithm we want to use to assign
assig.set_algorithm('bfw')

# since I haven't checked the parameters file, let's make sure convergence criteria is good
assig.max_iter = 1000
assig.rgap_target = 0.001

assig.execute() # we then execute the assignment

### Outputs


In [ ]:
# Convergence report is easy to see
import pandas as pd
convergence_report = assig.report()
convergence_report.head()

In [ ]:
volumes = assig.results()
volumes.head()

In [ ]:
# The link flows are easy to export.
# we do so for csv and AequilibraEData
assig.save_results('tutorial_ime_ano_base')

In [ ]:
# the skims are easy to get.

# The blended one are here
avg_skims = assigclass.results.skims

# The ones for the last iteration are here
last_skims = assigclass._aon_results.skims

# Assembling a single final skim file can be done like this
# We will want only the time for the last iteration and the distance averaged out for all iterations
kwargs = {'file_name': join(fldr,assg_fldr, 'skims.aem'),
          'zones': graph.num_zones,
          'matrix_names': ['time_final', 'distance_blended']}

# Create the matrix file
out_skims = AequilibraeMatrix()
out_skims.create_empty(**kwargs)
out_skims.index[:] = avg_skims.index[:]

# Transfer the data
 # The names of the skims are the name of the fields
out_skims.matrix['time_final'][:,:] = last_skims.matrix['free_flow_time'][:,:]
# It is CRITICAL to assign the matrix values using the [:,:]
out_skims.matrix['distance_blended'][:,:] = avg_skims.matrix['distance'][:,:]

out_skims.matrices.flush() # Make sure that all data went to the disk

# Export to OMX as well
out_skims.export(join(fldr,assg_fldr, 'skims.omx'))

    

# Trip distribution

### Calibration

We will calibrate synthetic gravity models using the skims for TIME that we just generated

In [ ]:
import numpy as np
from aequilibrae.distribution import GravityCalibration
from aequilibrae.matrix import AequilibraeMatrix

In [ ]:
# We need the demand
demand = AequilibraeMatrix()
demand.load(join(fldr, dt_fldr, 'demand.omx'))

# And the skims
imped = AequilibraeMatrix()
imped.load(join(fldr,assg_fldr, 'skims.aem'))

In [ ]:
# But before using the data, let's get some impedance for the intrazonals
# Let's assume it is 75% of the closest zone

# If we run the code below more than once, we will be overwriting the diagonal values with non-sensical data
# so let's zero it first
np.fill_diagonal(imped.matrix['time_final'], 0)

# We compute it with a little bit of NumPy magic
intrazonals = np.amin(imped.matrix['time_final'], where=imped.matrix['time_final']>0, initial=imped.matrix['time_final'].max(), axis=1)
intrazonals *= 0.75

# Then we fill in the impedance matrix
np.fill_diagonal(imped.matrix['time_final'], intrazonals)


In [ ]:
# We set the matrices forbeing used in computation
imped.computational_view(['time_final'])
demand.computational_view(['matrix'])

In [ ]:
from math import log10, floor
def plot_tlfd(demand, skim, name):
    import matplotlib.pyplot as plt
    b = floor(log10(skim.shape[0]) * 10)
    n, bins, patches = plt.hist(np.nan_to_num(skim.flatten(),0), bins = b, weights=np.nan_to_num(demand.flatten()), density=False, facecolor='g', alpha=0.75)

    plt.xlabel('Trip length')
    plt.ylabel('Probability')
    plt.title('Trip-length frequency distribution')
    plt.savefig(name, format="png")
    plt.clf()

In [ ]:
for function in ['power', 'expo']:
    model = GravityCalibration(matrix=demand, impedance=imped, function=function, nan_as_zero=True)
    model.calibrate()
    
    # we save the model
    model.model.save(join(fldr, dstr_fldr, f'{function}_model.mod'))
    
    # We save a trip length frequency distribution image
    plot_tlfd(model.result_matrix.matrix_view, imped.matrix_view,join(fldr, dstr_fldr, f'{function}_tfld.png') )
    
    # We can save the result of applying the model as well
    # we can also save the calibration report
    with open(join(fldr, dstr_fldr, f'{function}_convergence.log'), 'w') as otp:
        for r in  model.report:
            otp.write(r+'\n')

In [ ]:
# We save a trip length frequency distribution image
plot_tlfd(demand.matrix_view, imped.matrix_view,join(fldr, dstr_fldr, 'demand_tfld.png') )

# Forecast

* We create a set of *'future'* vectors using some random growth factors
* We apply the model for inverse power, as the TFLD seems to be a better fit for the actual one

In [ ]:
from aequilibrae.distribution import Ipf, GravityApplication, SyntheticGravityModel, Ipf
from aequilibrae.matrix import AequilibraeData, AequilibraeMatrix
import numpy as np

In [ ]:
# We compute the vectors from our matrix
mat = AequilibraeMatrix()

mat.load(join(fldr, dt_fldr, 'demand.omx'))
mat.computational_view()
origins = np.sum(mat.matrix_view, axis=1)
destinations = np.sum(mat.matrix_view, axis=0)

args = {'file_path':join(fldr,  frcst_fldr, 'synthetic_future_vector.aed'),
        "entries": mat.zones, 
        "field_names": ["origins", "destinations"],
    "data_types": [np.float64, np.float64], 
        "memory_mode": False}

vectors = AequilibraeData()
vectors.create_empty(**args)

vectors.index[:] =mat.index[:]

# Then grow them with some random growth between 0 and 10% - Plus balance them
vectors.origins[:] = origins * (1+ np.random.rand(vectors.entries)/10)
vectors.destinations[:] = destinations * (1+ np.random.rand(vectors.entries)/10)
vectors.destinations *= vectors.origins.sum()/vectors.destinations.sum()

In [ ]:
# Impedance 
imped = AequilibraeMatrix()
imped.load(join(fldr,assg_fldr, 'skims.aem'))
imped.computational_view(['time_final'])

# We want the main diagonal to be zero
np.fill_diagonal(imped.matrix_view, np.nan)

In [ ]:
for function in ['power', 'expo']:
    model = SyntheticGravityModel()
    model.load(join(fldr, dstr_fldr, f'{function}_model.mod'))

    outmatrix = join(fldr,frcst_fldr, f'demand_{function}_model.aem') 
    apply = GravityApplication()
    args = {"impedance": imped,
            "rows": vectors,
            "row_field": "origins",
            "model": model,
            "columns": vectors,
            "column_field": "destinations",
            "output": outmatrix,
            "nan_as_zero":True
            }

    gravity = GravityApplication(**args)
    gravity.apply()

    #We get the output matrix and save it to OMX too
    resm = AequilibraeMatrix()
    resm.load(outmatrix)
    resm.export(join(fldr,frcst_fldr, f'demand_{function}_model.omx'))

### We now run IPF for the future vectors

In [ ]:
demand = AequilibraeMatrix()
demand.load(join(fldr, dt_fldr, 'demand.omx'))
demand.computational_view()

args = {'matrix': demand,
        'rows': vectors,
        'columns': vectors,
        'column_field': "destinations",
        'row_field': "origins",
        'nan_as_zero': True}

ipf = Ipf(**args)
ipf.fit()

output = AequilibraeMatrix()
output.load(ipf.output.file_path)

output.export(join(fldr,frcst_fldr, 'demand_ipf.aem'))
output.export(join(fldr,frcst_fldr, 'demand_ipf.omx'))


# Future traffic assignment

In [ ]:
from aequilibrae.matrix import AequilibraeMatrix
from aequilibrae.paths import TrafficAssignment, TrafficClass
from aequilibrae import logger
import logging

In [ ]:
logger.info('\n\n\n TRAFFIC ASSIGNMENT FOR FUTURE YEAR')

In [ ]:
# from before
graph = project.network.graphs['c'] # we grab the graph for cars
graph.set_graph('free_flow_time') # let's say we want to minimize time
graph.set_skimming(['free_flow_time', 'distance']) # And will skim time and distance
graph.set_blocked_centroid_flows(False)

In [ ]:
# Let's use the IPF matrix
demand = AequilibraeMatrix()
demand.load(join(fldr, frcst_fldr, 'demand_ipf.omx'))
demand.computational_view() # There is only one matrix there, so don;t even worry about its core name

assig = TrafficAssignment()

# Creates the assignment class
assigclass = TrafficClass(graph, demand)

# The first thing to do is to add at list of traffic classes to be assigned
assig.add_class(assigclass)

assig.set_vdf("BPR")  # This is not case-sensitive # Then we set the volume delay function

assig.set_vdf_parameters({"alpha": "b", "beta": "power"}) # And its parameters

assig.set_capacity_field("capacity") # The capacity and free flow travel times as they exist in the graph
assig.set_time_field("free_flow_time")

# And the algorithm we want to use to assign
assig.set_algorithm('bfw')

# since I haven't checked the parameters file, let's make sure convergence criteria is good
assig.max_iter = 1000
assig.rgap_target = 0.001

assig.execute() # we then execute the assignment

In [ ]:

# The link flows are easy to export.
# we do so for csv and AequilibraEData
assig.save_results('tutorial_ime_futuro')

# the skims are easy to get.

# The blended one are here
avg_skims = assigclass.results.skims

# The ones for the last iteration are here
last_skims = assigclass._aon_results.skims

# Assembling a single final skim file can be done like this
# We will want only the time for the last iteration and the distance averaged out for all iterations
kwargs = {'file_name': join(fldr,ftr_fldr, 'future_skims.aem'),
          'zones': graph.num_zones,
          'matrix_names': ['time_final', 'distance_blended']}

# Create the matrix file
out_skims = AequilibraeMatrix()
out_skims.create_empty(**kwargs)
out_skims.index[:] = avg_skims.index[:]

# Transfer the data
 # The names of the skims are the name of the fields
out_skims.matrix['time_final'][:,:] = last_skims.matrix['free_flow_time'][:,:]
# It is CRITICAL to assign the matrix values using the [:,:]
out_skims.matrix['distance_blended'][:,:] = avg_skims.matrix['distance'][:,:]

out_skims.matrices.flush() # Make sure that all data went to the disk

# Export to OMX as well
out_skims.export(join(fldr,ftr_fldr, 'future_skims.omx'))


## Close the project

In [ ]:
project.close()

In [ ]:
a = ''
if a:
    print(1)

In [ ]:
volumes.reset_index(drop=True)

In [ ]:
volumes[~(volumes.Congested_Time_AB>5) & ~(volumes.Congested_Time_Max<8)]